In [4]:
"""
For a list of length INDS_LEN, select NUM_LIMIT element (x1, x2, x3, ..., xN).
While, for each element, a weight wi will be given. Wi = ((index(xi)+1) mod NUM_LIMIT) / NUM_LIMIT

Find max(x1*w1 * x2*w2 * x3*w3 *...* xN*xN), while sum(x1+x2+x3+...+xN)==100.

In this test, different from test 2, it will be harder to find the optimal solution. However, this test involved
repair mechanism instead of penalty function, which will treat each infeasible individual equally and repair it,
which achieves a significant improvement on performance and accuracy.
"""


from deap import base, creator, tools, algorithms
import random

INDS_LEN = 100
NUM_LIMIT = 10
SUM_CONSTRAINT = 100
POP_SIZE = 5000
GEN_NUM = 60

step_size = 1 / (INDS_LEN / NUM_LIMIT)
block = [round(i * step_size, 3) for i in range(1, INDS_LEN // NUM_LIMIT+1)]
RESULT_LIST = block * NUM_LIMIT

print(block)

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [5]:
def evaluate(individual):
    if not feasible(individual):
        repair(individual)
    
    r = 1
    min_len = min(len(individual), len(RESULT_LIST))
    for i in range(0, min_len):
        if individual[i] != 0:
            r *= individual[i] * RESULT_LIST[i]
    
    return r, 

def feasible(inidividual):
    f1 = sum(inidividual) == SUM_CONSTRAINT
    
    i = 0
    for ind in inidividual:
        if ind != 0:
            i += 1
    f2 = i == NUM_LIMIT
    
    return f1 and f2

def repair(individual):
    non_zero_indices = [i for i, x in enumerate(individual) if x > 0]
    zero_indices = [i for i, x in enumerate(individual) if x == 0]

    if len(non_zero_indices) > NUM_LIMIT:
        selected_indices = random.sample(non_zero_indices, NUM_LIMIT)
    else:
        selected_indices = non_zero_indices + random.sample(zero_indices, NUM_LIMIT - len(non_zero_indices))
    
    for i in range(len(individual)):
        if i not in selected_indices:
            individual[i] = 0
    
    if sum(individual) == 0:
        new_individual = create_individual()
        individual[:] = new_individual
    else:
        n = SUM_CONSTRAINT / sum(individual)
        for i in range(len(individual)):
            individual[i] = int(individual[i] * n)

        d = SUM_CONSTRAINT - sum(individual)
        if selected_indices:
            chosen_index = random.choice(selected_indices)
            individual[chosen_index] += d
            
def create_individual():
    individual = [0] * INDS_LEN

    chosen_positions = random.sample(range(INDS_LEN), NUM_LIMIT)

    remaining = SUM_CONSTRAINT
    for i in range(NUM_LIMIT-1):
        individual[chosen_positions[i]] = random.randint(1, remaining - (NUM_LIMIT - (i + 1)))
        remaining -= individual[chosen_positions[i]]
    individual[chosen_positions[-1]] = remaining

    return individual

In [6]:
# Setup DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary Algorithm
def main():
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=GEN_NUM, stats=stats, halloffame=hof)

    return hof[0]

# Run the GA
best_ind = main()


c:\Tool\Python3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Tool\Python3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	max                  
0  	5000  	(20346733.854720008,)
1  	2979  	(128986253.15143687,)
2  	2996  	(175107656.90879998,)
3  	2990  	(279884160.33177596,)
4  	3098  	(426720708.4139998,) 
5  	2988  	(635680199.7,)       
6  	2910  	(1182671964.5368316,)
7  	2928  	(1274691084.2265599,)
8  	3010  	(1840596847.1654406,)
9  	3098  	(1931350127.616,)    
10 	3049  	(2444542687.6416,)   
11 	3037  	(2707140046.6655993,)
12 	2980  	(3007933385.1839995,)
13 	2995  	(3222135796.239362,) 
14 	2880  	(3235424145.408,)    
15 	2993  	(3222135796.239362,) 
16 	2848  	(3550896340.9920006,)
17 	2941  	(4788921139.200001,) 
18 	3010  	(4585351680.0,)      
19 	2968  	(4800294826.9056,)   
20 	3083  	(6875422278.12,)     
21 	3034  	(6875422278.12,)     
22 	3001  	(6875422278.12,)     
23 	2971  	(6875422278.12,)     
24 	3033  	(6875422278.12,)     
25 	2938  	(7639358086.8,)      
26 	3046  	(7262105835.599999,) 
27 	2972  	(7716523320.0,)      
28 	3017  	(7639358086.800001,) 
29 	3015  

In [7]:
for i in range(0, len(best_ind), int(len(best_ind)/NUM_LIMIT)):
    print(best_ind[i:i + int(len(best_ind)/NUM_LIMIT)])
    
lst = [x for x in best_ind if x !=0]
print("Best Individual:", lst, "Fitness:", best_ind.fitness.values[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
Best Individual: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10] Fitness: 10000000000.0
